# MLDL-HW3 Demo Notebook

This notebook demonstrates the basic usage of the MLDL-HW3 codebase.

## Setup

In [ ]:
import sys
sys.path.insert(0, '..')

import torch
import numpy as np
import matplotlib.pyplot as plt

from src.models.neural_network import NeuralNetwork
from src.data.dataloader import create_dataloaders
from src.utils.training import train_epoch, evaluate

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

## Create Synthetic Data

In [ ]:
# Generate synthetic data for demonstration
np.random.seed(42)

# Parameters
n_samples = 1000
input_dim = 20
n_classes = 3

# Create synthetic features and labels
features = np.random.randn(n_samples, input_dim)
labels = np.random.randint(0, n_classes, n_samples)

print(f"Features shape: {features.shape}")
print(f"Labels shape: {labels.shape}")
print(f"Number of classes: {n_classes}")

## Create DataLoaders

In [ ]:
# Create train and test dataloaders
train_loader, test_loader = create_dataloaders(
    features, 
    labels, 
    batch_size=32, 
    test_size=0.2
)

print(f"Number of training batches: {len(train_loader)}")
print(f"Number of test batches: {len(test_loader)}")

## Initialize Model

In [ ]:
# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Create model
model = NeuralNetwork(
    input_dim=input_dim,
    hidden_dim=64,
    output_dim=n_classes,
    dropout_rate=0.3
).to(device)

print(f"\nModel architecture:")
print(model)
print(f"\nTotal parameters: {sum(p.numel() for p in model.parameters())}")

## Training

In [ ]:
import torch.nn as nn
import torch.optim as optim

# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
n_epochs = 5
train_losses = []
test_losses = []
train_accs = []
test_accs = []

for epoch in range(n_epochs):
    # Train
    train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)
    train_losses.append(train_loss)
    train_accs.append(train_acc)
    
    # Evaluate
    test_loss, test_acc = evaluate(model, test_loader, criterion, device)
    test_losses.append(test_loss)
    test_accs.append(test_acc)
    
    print(f"Epoch {epoch+1}/{n_epochs}:")
    print(f"  Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%")
    print(f"  Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.2f}%")

## Visualize Results

In [ ]:
# Plot training history
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

# Loss plot
ax1.plot(train_losses, label='Train Loss', marker='o')
ax1.plot(test_losses, label='Test Loss', marker='s')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.set_title('Training and Test Loss')
ax1.legend()
ax1.grid(True)

# Accuracy plot
ax2.plot(train_accs, label='Train Accuracy', marker='o')
ax2.plot(test_accs, label='Test Accuracy', marker='s')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Accuracy (%)')
ax2.set_title('Training and Test Accuracy')
ax2.legend()
ax2.grid(True)

plt.tight_layout()
plt.show()

## Save and Load Model

In [ ]:
# Save model
model_path = '../models/demo_model.pth'
model.save(model_path)
print(f"Model saved to: {model_path}")

# Create a new model and load weights
new_model = NeuralNetwork(
    input_dim=input_dim,
    hidden_dim=64,
    output_dim=n_classes
).to(device)

new_model.load(model_path)
print("Model loaded successfully!")

# Verify loaded model has same accuracy
test_loss, test_acc = evaluate(new_model, test_loader, criterion, device)
print(f"Loaded model test accuracy: {test_acc:.2f}%")

## Conclusion

This notebook demonstrated:
1. Creating synthetic data
2. Building dataloaders
3. Initializing and training a neural network
4. Evaluating model performance
5. Visualizing training history
6. Saving and loading models

You can modify this notebook to work with your own datasets and experiments!